# Introduction Inteactive Map for Python 104


Folium เป็นไลบรารีใน Python ที่ใช้สำหรับการสร้างแผนที่เชิงโต้ตอบ (Inteactive Map) โดยทำงานร่วมกับข้อมูลเชิงพื้นที่จาก GeoDataFrame หรือ GeoJSON และสามารถแสดงผลบนแผนที่ที่เป็นที่นิยมอย่าง OpenStreetMap หรือ อื่นๆ ภายใต้ระบบพิกัด WGS 84 (EPSG:4326) 

In [1]:
import requests
import geopandas as gpd
import pandas as pd
import folium
import matplotlib.pyplot as plt
from shapely.geometry import Point,LineString, Polygon

 ตัวอย่างข้อมูล

In [2]:
# Example Data
import requests
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point,LineString, Polygon
import matplotlib.pyplot as plt

################# ขั้นตอนที่ 1 Load ข้อมูลมาใส่ Dataframe

# service แบบ geometry(x,y) --> point ตำแหน่งปลอดภัยจากแผ่นดินถล่ม :: https://data.dmr.go.th/en_AU/dataset/safe_landslide_area?activity_id=0f5a2afc-a024-4243-b5a9-72570f363ba8
url_point = "https://gisportal.dmr.go.th/arcgis/rest/services/HAZARD/%E0%B8%95%E0%B8%B3%E0%B9%81%E0%B8%AB%E0%B8%99%E0%B9%88%E0%B8%87%E0%B8%9B%E0%B8%A5%E0%B8%AD%E0%B8%94%E0%B8%A0%E0%B8%B1%E0%B8%A2%E0%B8%88%E0%B8%B2%E0%B8%81%E0%B9%81%E0%B8%9C%E0%B9%88%E0%B8%99%E0%B8%94%E0%B8%B4%E0%B8%99%E0%B8%96%E0%B8%A5%E0%B9%88%E0%B8%A1/MapServer/0/query"

# service แบบ paths --> linestring ชั้นข้อมูลแผนที่เส้นทางหลวงชนบท :: https://datagov.mot.go.th/dataset/transportgis
url_line = "https://giportal.mot.go.th/arcgis/rest/services/Hosted/%E0%B9%80%E0%B8%AA%E0%B9%89%E0%B8%99%E0%B8%97%E0%B8%B2%E0%B8%87%E0%B8%AB%E0%B8%A5%E0%B8%A7%E0%B8%87%E0%B8%8A%E0%B8%99%E0%B8%9A%E0%B8%97_%E0%B8%97%E0%B8%8A/FeatureServer/0/query"

# service แบบ ring --> polygon พื้นที่มีโอกาสเกิดแผ่นดินถล่ม :: https://data.dmr.go.th/eu/dataset/landslide_susceptibility
url_polygon = "https://gisportal.dmr.go.th/arcgis/rest/services/HAZARD/LANDSLIDE_SUSCEPTIBILITY/MapServer/0/query"

# กำหนดพารามิเตอร์สำหรับการ query
params = {
    'where': '1=1',  # ดึงข้อมูลทั้งหมด
    'outFields': '*',  # ดึงทุกฟิลด์
    'f': 'json',  # ระบุว่าเราต้องการข้อมูลเป็น JSON
    'resultRecordCount': 2000,  # จำนวนแถวที่ต้องการดึงแต่ละครั้ง
    'resultOffset': 0,  # ชุดข้อมูลเริ่มต้น
}

# ฟังก์ชันดึงข้อมูลทั้งหมดจาก ArcGIS REST API
def fetch_all_data(url, params):
    data = []
    offset = 0
    while True:
        params['resultOffset'] = offset  # ปรับ resultOffset สำหรับแต่ละการดึงข้อมูล
        
        # ส่ง request ไปยัง API
        response = requests.get(url, params=params)
        response_json = response.json()
        
        # ตรวจสอบว่ามีฟีเจอร์ในผลลัพธ์หรือไม่
        if 'features' in response_json and response_json['features']:
            data.extend(response_json['features'])  # เพิ่มข้อมูลใหม่ที่ดึงมา
            offset += params['resultRecordCount']  # เพิ่ม offset เพื่อดึงข้อมูลชุดถัดไป
        else:
            break  # ถ้าไม่มีฟีเจอร์ในผลลัพธ์ ให้หยุดการดึงข้อมูล
    return data

# ดึงข้อมูลจากบริการ ArcGIS REST API
data_point = fetch_all_data(url_point, params.copy())
data_line = fetch_all_data(url_line, params.copy())
data_polygon = fetch_all_data(url_polygon, params.copy())


################# ขั้นตอนที่ 2 แปลงเป็น Geodataframe

# ฟังก์ชันแปลงเป็น GeoDataFrame
def convert_to_geodataframe(geometry_type, features):
    geometries = []
    attributes = []
    
    for feature in features:
        geom = feature.get('geometry')
        attr = feature.get('attributes')
        if geom:  # ตรวจสอบว่ามี geometry หรือไม่
            if geometry_type == "esriGeometryPoint":
                coords = geom.get('x'), geom.get('y')
                geometries.append(Point(coords))
            elif geometry_type == "esriGeometryPolyline":
                coords = geom.get('paths', [])[0]
                geometries.append(LineString(coords))
            elif geometry_type == "esriGeometryPolygon":
                coords = geom.get('rings', [])[0]
                geometries.append(Polygon(coords))
            attributes.append(attr)  # เก็บ attributes
    
    if geometries and attributes:
        # สร้าง DataFrame จาก attributes และ GeoDataFrame จาก geometries
        df_attributes = pd.DataFrame(attributes)
        gdf = gpd.GeoDataFrame(df_attributes, geometry=geometries)
    else:
        print("No geometries or attributes found.")
        gdf = None
    
    return gdf


# แปลงข้อมูลเป็น GeoDataFrame และตั้งค่า EPSG setting geometryType , features  >> Create a GeoDataFrame >> Set the CRS (Coordinate Reference System)
gdf_point = convert_to_geodataframe("esriGeometryPoint", data_point).set_crs(epsg=32647, inplace=True)
gdf_line = convert_to_geodataframe("esriGeometryPolyline", data_line).set_crs(epsg=4326, inplace=True)
gdf_polygon = convert_to_geodataframe("esriGeometryPolygon", data_polygon).set_crs(epsg=32647, inplace=True)



#### 1. การสร้างแผนที่พื้นฐาน

In [5]:
import folium
from folium.plugins import MeasureControl

# สร้างแผนที่พื้นฐานที่พิกัดศูนย์กลางของประเทศไทย
m1 = folium.Map(
    location=[13.75, 100.5],  # พิกัดศูนย์กลางของประเทศไทย
    zoom_start=6  # ระดับการซูมที่เหมาะสมสำหรับประเทศไทย
)


In [6]:
import folium
from folium.plugins import MeasureControl

# สร้างแผนที่พื้นฐานที่พิกัดศูนย์กลางของประเทศไทย
m1 = folium.Map(
    location=[13.75, 100.5],  # พิกัดศูนย์กลางของประเทศไทย
    zoom_start=6  # ระดับการซูมที่เหมาะสมสำหรับประเทศไทย
)


#### 2. การนำเสนอข้อมูล geodataframe มาแสดงบนแผนที่

In [7]:
# แสดงข้อมูลจุด

# gdf_point.dtypes
gdf_point_deploy = gdf_point.copy().to_crs(epsg=4326)
gdf_point_deploy = gdf_point_deploy.head(50)

# แสดง Points บนแผนที่
for _, row in gdf_point_deploy.iterrows():
    # ดึงค่าพิกัดจาก geometry
    coords = [row.geometry.y, row.geometry.x]
    # เพิ่ม Marker ลงในแผนที่
    popup_text = (
        f"OBJECTID: {row['OBJECTID']}<br>"
        #f"SAFE_ID: {row['SAFE_ID']}<br>"
        #f"UTM_E: {row['UTM_E']}<br>"
        #f"UTM_N: {row['UTM_N']}<br>"
        #f"ZONE: {row['ZONE']}<br>"
        f"MOO: {row['MOO']}<br>"
        f"PLACE: {row['PLACE']}<br>"
        f"VILLAGE: {row['VILLAGE']}<br>"
        f"TAMBON: {row['TAMBON']}<br>"
        f"DISTRICT: {row['DISTRICT']}<br>"
        f"PROVINCE: {row['PROVINCE']}<br>"
        f"YEAR_MAP: {row['YEAR_MAP']}<br>"
        #f"REMARK: {row['REMARK']}<br>"
        #f"REMARK_2: {row['REMARK_2']}"
    )
    folium.Marker(
        location=coords,
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m1)

In [8]:
# แสดงข้อมูลเส้น

gdf_line_deploy = gdf_line.copy().to_crs(epsg=4326)
gdf_line_deploy = gdf_line_deploy.head(50)
#gdf_line_deploy.dtypes

# แสดง Lines บนแผนที่
for _, row in gdf_line_deploy.iterrows():
    # ดึงค่าพิกัดจาก geometry
    coords = [(y, x) for x, y in row['geometry'].coords]
    # เพิ่ม Marker ลงในแผนที่
    popup_text = (
        f"objectid: {row['objectid']}<br>"
        f"route_name: {row['route_name']}<br>"
        f"road_code: {row['road_code']}<br>"
        f"km_start: {row['km_start']}<br>"
        f"km_end: {row['km_end']}<br>"
        f"length_drr: {row['length_drr']}<br>"
    )
    folium.PolyLine(
        locations=coords,
        color='blue',
        weight=5,
        opacity=0.8,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m1)

In [9]:
# แสดงข้อมูลพื้นที่
from shapely.geometry import mapping

gdf_polygon_deploy = gdf_polygon.copy().to_crs(epsg=4326)
gdf_polygon_deploy = gdf_polygon_deploy.head(500)
#gdf_polygon_deploy.dtypes

# แสดง Polygon บนแผนที่
for _, row in gdf_polygon_deploy.iterrows():
    # ดึงค่าพิกัดจาก geometry
    geo_json = mapping(row['geometry'])
    # เพิ่ม Marker ลงในแผนที่
    popup_text = (
        f"OBJECTID: {row['OBJECTID']}<br>"
        f"Level_E: {row['Level_E']}<br>"
        f"Desc_T: {row['Desc_T']}<br>"
        f"Area: {row['Shape.STArea()']}<br>"
    )
    folium.GeoJson(
        geo_json,
        style_function=lambda x: {
            'color': 'red',
            'fillColor': 'red',
            'fillOpacity': 0.4
        },
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m1)

#### 3. การแสดง องค์ประกอบแผนที่

In [15]:
# เตรียมข้อมูล

# ดึงข้อมูล เฉพาะจังหวัดที่ต้องการ
gdf_point_deploy2 = gdf_point.copy().to_crs(epsg=4326)
gdf_point_deploy2 = gdf_point_deploy2[gdf_point_deploy2['PROVINCE'] == 'กาญจนบุรี']


# ดึงข้อมูล เฉพาะจังหวัดขอนแก่น โดยจะมีรหัสสายทางขึ้นด้วย ขก.
gdf_line_deploy2 = gdf_line.copy().to_crs(epsg=4326)
gdf_line_deploy2 = gdf_line_deploy2[gdf_line_deploy2['road_code'].str.contains('กจ.', na=False)]


# ดึงข้อมูล polygon ที่อยู่ในรัศมีของสายทางใน ที่ระยะ 2 กิโลเมตร อย่าลืมแปลงเป็น 32647 ก่อนนะ
gdf_polygon_deploy2 = gdf_polygon.copy().to_crs(epsg=4326)
gdf_polygon_deploy2 = gpd.sjoin(
    gdf_polygon_deploy2.to_crs(epsg=32647),
    gdf_line_deploy2.to_crs(epsg=32647).assign(geometry=lambda x:x['geometry'].buffer(2000)),
    how='inner' , op='intersects'
).to_crs(epsg=4326)

gdf_polygon_deploy2 = gdf_polygon_deploy2.drop_duplicates(subset=['geometry']).reset_index(drop=True)


In [11]:
# คำนวณขอบเขตของข้อมูล (bounding box) จาก GeoDataFrame
bounds = gdf_polygon_deploy2.total_bounds  # คืนค่าขอบเขตเป็น [minx, miny, maxx, maxy]

# สร้างแผนที่และตั้งค่าพิกัดและการซูมให้พอดีกับข้อมูล
m2 = folium.Map()

# ปรับการแสดงผลของแผนที่ให้อยู่ในขอบเขตของข้อมูล
m2.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])

In [12]:
import random

# ดึงค่าที่ไม่ซ้ำในคอลัมน์ DISTRICT
unique_gdf_point_deploy2 = gdf_point_deploy2['DISTRICT'].unique()
# สร้าง dictionary ของสีสุ่มสำหรับแต่ละ DISTRICT
district_colors = {'ศรีสวัสดิ์': 'red',
 'เลาขวัญ': 'orange',
 'ทองผาภูมิ': 'lightblue',
 'ไทรโยค': 'gray',
 'เมืองกาญจนบุรี': 'pink',
 'สังขละบุรี': 'blue',
 'บ่อพลอย': 'green',
 'หนองปรือ': 'lightgreen'}


# แสดง Points บนแผนที่
for _, row in gdf_point_deploy2.iterrows():
    # ดึงค่าพิกัดจาก geometry
    coords = [row.geometry.y, row.geometry.x]
    # กำหนดสีตาม DISTRICT
    icon_color = district_colors.get(row['DISTRICT'])  # ใช้ 'gray' เป็นค่าเริ่มต้นหาก DISTRICT ไม่พบใน dictionary
    # เพิ่ม Marker ลงในแผนที่
    popup_text = (
        f"OBJECTID: {row['OBJECTID']}<br>"
        f"MOO: {row['MOO']}<br>"
        f"PLACE: {row['PLACE']}<br>"
        f"VILLAGE: {row['VILLAGE']}<br>"
        f"TAMBON: {row['TAMBON']}<br>"
        f"DISTRICT: {row['DISTRICT']}<br>"
        f"PROVINCE: {row['PROVINCE']}<br>"
        f"YEAR_MAP: {row['YEAR_MAP']}<br>"
    )
    folium.Marker(
        location=coords,
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color=icon_color, icon='info-sign')
    ).add_to(m2)



# แสดง Lines บนแผนที่
for _, row in gdf_line_deploy2.iterrows():
    # ดึงค่าพิกัดจาก geometry
    coords = [(y, x) for x, y in row['geometry'].coords]
    # เพิ่ม Marker ลงในแผนที่
    popup_text = (
        f"objectid: {row['objectid']}<br>"
        f"route_name: {row['route_name']}<br>"
        f"road_code: {row['road_code']}<br>"
        f"km_start: {row['km_start']}<br>"
        f"km_end: {row['km_end']}<br>"
        f"length_drr: {row['length_drr']}<br>"
    )
    folium.PolyLine(
        locations=coords,
        color='blue',
        weight=5,
        opacity=0.8,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m2)




# ดึงค่าที่ไม่ซ้ำในคอลัมน์ DISTRICT
unique_gdf_polygon_deploy2 = gdf_polygon_deploy2['Level_E'].unique()
# สร้าง dictionary ของสีสุ่มสำหรับแต่ละ DISTRICT
level_colors = {'Very High': 'darkred',
 'High': 'red',
 'Medium': 'orange',
 'Low': 'gold',
 'Very Low': 'lightyellow'}

# แสดง Polygon บนแผนที่
for _, row in gdf_polygon_deploy2.iterrows():
    # ดึงค่าพิกัดจาก geometry
    geo_json = mapping(row['geometry'])

    # กำหนดสีตามค่า Level_E
    fill_color = level_colors.get(row['Level_E'], 'gray')

    # เพิ่ม Marker ลงในแผนที่
    popup_text = (
        f"OBJECTID: {row['OBJECTID']}<br>"
        f"Level_E: {row['Level_E']}<br>"
        f"Desc_T: {row['Desc_T']}<br>"
        f"Area: {row['Shape.STArea()']}<br>"
    )
    folium.GeoJson(
        geo_json,
        style_function=lambda feature, row=row: {
            'color': level_colors.get(row['Level_E'], 'gray'),  # ขอบสี
            'fillColor': level_colors.get(row['Level_E'], 'gray'),  # สีพื้น
            'fillOpacity': 0.4
        },
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m2)



In [14]:
m2.save("map.html")